In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras.utils as image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
import sklearn
from sklearn.model_selection import train_test_split
from statistics import mean

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_Train = np.load('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/X_Train.npy', mmap_mode = 'r')
Y_Train = np.load('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Y_Train.npy', mmap_mode = 'r')

In [ ]:
X_Val = np.load('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/X_Val.npy', mmap_mode = 'r')
Y_Val = np.load('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/Y_Val.npy', mmap_mode = 'r')


In [ ]:
batch_size = 64
nb_epoch = 50
steps_per_epoch = np.ceil(X_Train.shape[0]/batch_size)
validation_steps = np.ceil(X_Val.shape[0]/batch_size)
print(steps_per_epoch)
print(validation_steps)

381.0
147.0


In [ ]:
from keras.applications import VGG16

In [ ]:
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(200,150, 3))


In [ ]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

In [ ]:
num_classes = 28  # Types of Genres


In [ ]:
# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

In [ ]:
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 1024)              12583936  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 28)                28700     
                                                                 
Total params: 27,327,324
Trainable params: 19,692,060
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# construct the training image generator for data augmentation
datagen = ImageDataGenerator(rotation_range=20,
                             zoom_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.15,
                             horizontal_flip=True, fill_mode="nearest",
                             validation_split=0.2)

In [ ]:
history = model.fit(datagen.flow_from_dataframe(X_Train, Y_Train, batch_size=batch_size),
                    validation_data=(X_Val, Y_Val),
                    steps_per_epoch=steps_per_epoch,
                    epochs=nb_epoch,
                    verbose=1)

model.save('/content/drive/MyDrive/Summer_2023/Models/Poster_Models/finetuned-vgg16-model.h5')